In [6]:
import pandas as pd

# Load the dataset
file_path = 'IMDB_Dataset.xlsx'
data = pd.read_excel(file_path)

# Display the first few rows of the dataset
print(data.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [7]:
import re
from sklearn.preprocessing import LabelEncoder

# Function to clean the text
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert text to lowercase
    text = text.lower()
    return text

# Apply the cleaning function to the review column
data['cleaned_review'] = data['review'].apply(clean_text)

# Convert sentiment labels to binary values
label_encoder = LabelEncoder()
data['sentiment'] = label_encoder.fit_transform(data['sentiment'])

# Display the first few rows of the cleaned dataset
print(data[['cleaned_review', 'sentiment']].head())


                                      cleaned_review  sentiment
0  one of the other reviewers has mentioned that ...          1
1  a wonderful little production the filming tech...          1
2  i thought this was a wonderful way to spend ti...          1
3  basically theres a family where a little boy j...          0
4  petter matteis love in the time of money is a ...          1


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_review'], data['sentiment'], test_size=0.2, random_state=42)

# Vectorize the text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [10]:
# Train a Logistic Regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Evaluate the model
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{classification_rep}')


Accuracy: 0.8936
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      4961
           1       0.88      0.91      0.90      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [11]:
def predict_sentiment(review):
    # Clean the review
    cleaned_review = clean_text(review)
    # Vectorize the review
    vectorized_review = tfidf_vectorizer.transform([cleaned_review])
    # Predict the sentiment
    prediction = model.predict(vectorized_review)
    # Convert prediction to sentiment label
    sentiment = label_encoder.inverse_transform(prediction)[0]
    return sentiment

# Example new reviews
new_reviews = [
    "The movie was fantastic! I loved the performances and the story was gripping.",
    "I didn't like the movie. The plot was too predictable and the acting was subpar."
]

# Predict sentiments for the new reviews
predicted_sentiments = [predict_sentiment(review) for review in new_reviews]
print(predicted_sentiments)


[1, 0]
